In [ ]:
from citipy import citipy
import json
import numpy as np
import pandas as pd
import time

from config import weather_api_key
from config import weather_api_key1

In [46]:
# Create a set of 2000 random latitude and longitude combinations.
num_locs = 2000

lats = np.random.uniform(low=-90.000, high=90.000, size=num_locs)
lngs = np.random.uniform(low=-180.000, high=180.000, size=num_locs)

lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [47]:
# Create lists to store cities and countries.
cities = []

# Loop over the coordinates.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # Only append if it is a new city.
    if city not in cities:
        cities.append(city)
    
# Sanity checks:
print (f"Initial count of cities: {len(cities)}")

Initial count of cities: 761


In [48]:
# Base URL for the OpenWeather API call.
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key1

print("-------------------------------------------------------------------------------")
print(f"Beginning Data Retrieval: Will attempt to find weather data for {len(cities)} cities")
print("-------------------------------------------------------------------------------\n")

# Create an empty list to hold the weather data.
city_data = []

## WARNING! The code below will take several mins to run, as we will throttle the API requests
# to 60 per min. This is because openweatherAPI will otherwise block the account if the rate limit
# is exceeded.

# If throttling is not desired, simply comment out the line: time.sleep(sleep_time)


# Initialize variables to keep track of rate limiting.
start_time = int(time.time())
current_time = start_time

# Loop over all cities.
for i, city in enumerate(cities):

    # Log every 60th record. In addition, introduce a sleep for the next batch so
    # we do not exceed the rate limit of openweatherAPI.
    if i % 60 == 0:
        print (f"Retrieving data for record {i} | {city}")
        
        if i > 0:
            # Get current time.
            current_time = int(time.time())
        
            # If we have made 60 calls and 1 min hasn't passed, we will sleep here.
            if current_time - start_time < 60:
                sleep_time = 60 - (current_time - start_time)
                print (f"Introducting a sleep of {sleep_time} seconds!")
                time.sleep(sleep_time) # <------ COMMENT OUT THIS LINE IF THROTTLING IS NOT DESIRED.
        
            # Get the new start time.
            start_time = int(time.time())
        
    
    # Create endpoint URL with each city.
    city_url = base_url + "&q=" + city.replace(" ","+")
    
    try:
        city_weather = requests.get(city_url).json()
        
        # The code below was used to check openweatherAPI response. It is commented out now.
        # rsp = json.dumps(city_weather, indent=4)
        # print(rsp)
        
        # Parse out the needed data.
        lat = city_weather["coord"]["lat"]
        lng = city_weather["coord"]["lon"]
        max_temp = city_weather["main"]["temp_max"]
        humidity = city_weather["main"]["humidity"]
        clouds = city_weather["clouds"]["all"]
        wind = city_weather["wind"]["speed"]
        country = city_weather["sys"]["country"]
        desc = city_weather['weather'][0]['description']

        # Append the city information into city_data list.
        city_data.append({'City': city.title(),
                          'Lat': lat,
                          'Lng': lng,
                          'Max Temp': max_temp,
                          'Humidity': humidity,
                          'Cloudiness': clouds,
                          'Wind Speed': wind,
                          'Country': country,
                          'Current Description': desc})

    except Exception as e:
        print(f"Failed attempt to find city: {city}, openweather response: {city_weather['message']}")
        pass

print("\n-------------------------------------------------------------------------------")
print(f"Data Retrieval Complete: Number of cities with valid weather data: {len(city_data)}")
print("-------------------------------------------------------------------------------")

-------------------------------------------------------------------------------
Beginning Data Retrieval: Will attempt to find weather data for 761 cities
-------------------------------------------------------------------------------

Retrieving data for record 0 | busselton
Failed attempt to find city: barentsburg, openweather response: city not found
Failed attempt to find city: taolanaro, openweather response: city not found
Failed attempt to find city: yomitan, openweather response: city not found
Failed attempt to find city: warqla, openweather response: city not found
Failed attempt to find city: meyungs, openweather response: city not found
Failed attempt to find city: saleaula, openweather response: city not found
Retrieving data for record 60 | punta arenas
Introducting a sleep of 49 seconds!
Failed attempt to find city: tuggurt, openweather response: city not found
Failed attempt to find city: grimari, openweather response: city not found
Failed attempt to find city: karkara

In [50]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 
                'Cloudiness', 'Wind Speed', 'Current Description']

city_data_df = city_data_df[column_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,55.09,82,90,8.19,overcast clouds
1,Puerto Ayora,EC,-0.7393,-90.3518,78.76,88,56,5.99,broken clouds
2,Upernavik,GL,72.7868,-56.1549,31.21,94,94,16.31,overcast clouds
3,Atuona,PF,-9.8000,-139.0333,78.66,71,14,11.01,few clouds
4,Tiarei,PF,-17.5333,-149.3333,75.22,77,43,12.33,light rain


In [52]:
import os

# Save to CSV.
output_data_file = os.path.join('Weather_Database', 'WeatherPy_Database.csv')
city_data_df.to_csv(output_data_file, index_label="City_ID")